In [382]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeamsInQuery
import sqlAnswers

In [597]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [598]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    print(query)
    return query


Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [639]:

query = preProcessQuery("Wie Arsenal Liverpool?")
query = preProcessQuery("Wie viel wiegt Thomas Müller ?")
query = preProcessQuery("in Welchem Verein spielt Mesut Özil ?")
#queryRefined = [getPlayersInQuery(query), getLeaguesInQuery(query),getTeamsInQuery(query),["2008/2009"]]

season = ["2008/2009"]

queryRefined = [getPlayersInQuery(query), getLeaguesInQuery(query),getTeamsInQuery(query),season]




print(queryRefined)

import importlib
importlib.reload(sqlAnswers)

answers = sqlAnswers.getAnswers()

Wie Arsenal Liverpool?
Wie viel wiegt Thomas Mueller ?
in Welchem Verein spielt Mesut Oezil ?
[[7327], [], [], ['2008/2009']]


In [640]:
queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3])]
queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3])]
possibleAnswers = [] 
#contains index with possible ansers which could match the query depending
#on number of player occurances etc

for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
    sqlMatches = True;
    for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
        if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
            sqlMatches = False
    if(sqlMatches):
        possibleAnswers.append(x)
        

print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
print(possibleAnswers)

In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[4]


In [641]:
#print(sqlAnswers.getTeamApiIDByID(3459))
#print(sqlAnswers.getMatchResultsAgainst(3459, 3462,"2015/2016"))

for index in possibleAnswers:
    print(answers[index][4](queryRefined))
    

Der Spieler spielte in Saison 2008/2009 bei Arsenal.


In [582]:
#print(sqlAnswers.getTeamApiIDByPlayerID(6556, '2012/2013'))

print(sqlAnswers.getHomeMatchesOfPlayer(27299, '2015/2016'))
#getHomeMatchesOfPlayer(getPlayerAPIIdById(player_id),season)

[(9951,), (9960,), (9978,), (9996,), (10014,), (10041,), (10077,), (10095,), (10113,), (10131,), (10149,), (10158,), (10176,), (10194,), (10203,), (10221,), (10239,)]


In [619]:
def getTeamMatches():
    import sqlite3
    conn = sqlite3.connect('database.sqlite')
    c = conn.cursor()
    queryText = 'SELECT league_id FROM match WHERE home_team_api_id = 10000 AND away_team_api_id = 9994'
    queryText = 'SELECT team_api_id FROM team WHERE id = 15617'
    #Bayern münchen hat die api id von 9823, die normale ID wäre 15617
    
    #wenn keine season angegeben wird, evtl die letzte nehmen? weil es gibt ja auch spielerwechsel
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 OR away_team_api_id = 9823 AND season = 2008/2009 '
    queryText = 'SELECT * FROM match WHERE match.id = 7809 AND season = "2008/2009"'
    
    
    
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 OR away_team_api_id = 9823' #liefert alle spiele von bayern münchen
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 OR away_team_api_id = 9823 AND season = "2015/2016"'
    
    queryText = 'SELECT id FROM match WHERE home_team_api_id = 9823 '#OR away_team_api_id = 9823 AND season = "2015/2016"'
    
    
    #10239 hatte bayern ein Heimspiel
    #spieler 1 war da 27299(api) / 6556, das heißt manuel Neuer war im Tor
    #queryText = 'SELECT away_player_1 FROM match WHERE id = 10189'
    queryText = 'SELECT home_team_api_id FROM match WHERE id = 7838'  #hier soll wohl schalke ein heimspiel haben
    
    queryText = 'SELECT * FROM team WHERE team_api_id = 10189'
    
    
    team_api_id1 = sqlAnswers.getTeamApiIDByID(3462)
    team_api_id2 = sqlAnswers.getTeamApiIDByID(3459)
    queryText = 'SELECT home_team_goal FROM match WHERE home_team_api_id = '+str(team_api_id1)+' AND away_team_api_id = '+str(team_api_id2) + ' AND season = "2015/2016"'
    #season = "season = '2015/2016'"
    #player_api_id = 27299
    #queryText = 'SELECT id FROM match WHERE '+season+' AND  home_player_1 =' + str(player_api_id) + ' OR home_player_2 =' + str(player_api_id) + ' OR home_player_3 =' + str(player_api_id)+ ' OR home_player_4 =' + str(player_api_id)+ ' OR home_player_5 =' + str(player_api_id)+ ' OR home_player_6 =' + str(player_api_id)+ ' OR home_player_7 =' + str(player_api_id)+ ' OR home_player_8 =' + str(player_api_id)+ ' OR home_player_9 =' + str(player_api_id)+ ' OR home_player_10 =' + str(player_api_id)+ ' OR home_player_11 =' + str(player_api_id)
    
    
    c.execute(queryText)
    result = c.fetchall()
    conn.close()
    return result
print(getTeamMatches())
#print(sqlAnswers.getTeamApiIDByName("Liverpool"))

[(3,)]
